In [1]:
import numpy as np
import pandas as pd

In [55]:
inp_dim = 2
hidden_dim = 2
out_dim = 1

#### Initialize a network

In [56]:

def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[np.random.rand() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[np.random.rand() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [57]:
# shallow neural network
shallowNN = initialize_network(inp_dim,hidden_dim,out_dim)
for i,layer in enumerate(shallowNN):
    print(f"layer {i+1} weights {layer} \n")
    

layer 1 weights [{'weights': [0.35893298249296823, 0.12751404719790382, 0.2931806308922843]}, {'weights': [0.09738219822767868, 0.5554577825110106, 0.2530447863002404]}] 

layer 2 weights [{'weights': [0.06323017838176448, 0.9896621588700363, 0.7672490588081572]}] 



#### Forward Propogation
Divided into 3 steps :
1. Activation of the neuron
2. Pass the neuron data to next layer
3. Assemble all and do forward propogation

#### Activation of the neuron

If we think clearly, calculating a one neuron value (next layer) based on it's last layer inputs is the like the weighted sum of the inputs i.e w<sub>i</sub>* h<sub>l-1</sub> + b 

In [58]:
def activate(wgts,inputs):
    bias = wgts[-1]
    activation = 0
    for i in range(len(wgts)-1):
        activation+=wgts[i]*inputs[i]
    activation+=bias
    
    return activation

In [59]:
# random input initialization
input_init = []
for _ in range(inp_dim):
    input_init.append(np.random.rand())

# calculating for hidden layer 1
output_layer_input = list()
for cell in shallowNN[0]:
    output_layer_input.append(
                                activate(cell['weights']
                                ,input_init)
                             )

#### Neuron data transfer to next layer

In [60]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

activation = list(map(lambda k: sigmoid(k),output_layer_input))
print(f"next layer input is {activation}")

next layer input is [0.6158594358797047, 0.664506429804911]


#### Forward Propogation 

In [61]:
def forward_propagate(network,inp):
    inputs = inp
    for layer in network:
        new_inputs = []
        for neuron in layer:
            out = activate(neuron['weights'],inputs)
            neuron['output'] = sigmoid(out)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [62]:
output = forward_propagate(shallowNN,input_init)
print(f"last layer output {output}")

last layer output [0.8121172805875845]


#### Back Propagation
Divided into 2 section:
1. Derivative calculation
2. Backpropagate the error

#### Derivative Calculation

In [63]:
def calculate_derivative(x):
    """
    d(sigmoid) = x*(1-x)
    """
    return x*(1-x)

#### Backpropagate the error
1. calculate the error for each output neuron
2. consider above as an input to backpropagate in the network the same way we did for forward propagate.

3. err = (out-err)*calculate_derivative(out) --> for last layer
4. err = (wgt<sub>k</sub>* err<sub>j</sub>)*calculate_derivative(out) --> for hidden layer


In [74]:
def backpropagate(network,expected):
    for i in range(len(network)-1,-1,-1):
        layer = network[i]
        errors = list()
        if i==len(network)-1:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(neuron['output']-expected[j])
        else:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i+1]:
                    error+=(neuron['weights'][j]*(neuron['delta']))
                errors.append(error)
        
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j]*calculate_derivative(neuron['output'])
    return network

In [75]:
backpropagate(shallowNN,[1])

[[{'weights': [0.35893298249296823, 0.12751404719790382, 0.2931806308922843],
   'output': 0.6158594358797047,
   'delta': -0.0004288334000987567},
  {'weights': [0.09738219822767868, 0.5554577825110106, 0.2530447863002404],
   'output': 0.664506429804911,
   'delta': -0.006325032774982391}],
 [{'weights': [0.06323017838176448, 0.9896621588700363, 0.7672490588081572],
   'output': 0.8121172805875845,
   'delta': -0.02866767199300915}]]

#### Train the network
Let's just use the backprop error to train the network
1. Update weights
2. Train Network

In [ ]:
def update_nn_weights(network, sample, lr):
    for i in range(len(network)):
        inputs = sample[:-1] #skipping last index b/c its label
        if i!=0:
            inputs = [neuron['output'] for neuron in network[i-1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j]-=lr*inputs[j]*neuron['delta']
        neuron['weights'][-1]-=lr*neuron['delta']
        

In [ ]:
## still in progress